In [1]:
import os 

In [2]:
%pwd

'e:\\GEN AI\\Rag-gen-ai-project\\Medical-chatbot\\research'

In [3]:
os.chdir("../")

In [4]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
def load_pdf_files(data): 
    loader=DirectoryLoader(
        data, 
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents=loader.load()
    return documents

In [7]:
def text_splitter(data): 
    text_split=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
    text_chunk=text_split.split_documents(data)
    return text_chunk

In [10]:
extracted_data=load_pdf_files("Data/")
text_chunk=text_splitter(extracted_data)

In [17]:
from langchain_community.embeddings import HuggingFaceEmbeddings 

In [20]:
def download_embeddings(): 
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [21]:
embeddings=download_embeddings()

e:\anaconda\envs\chatbot-rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
from langchain.vectorstores import Chroma 

dir="vector_database/"

vector_database=Chroma.from_documents(
    documents=text_chunk, 
    persist_directory=dir, 
    embedding=embeddings
)

In [24]:
doc_serach=Chroma(
    persist_directory=dir, 
    embedding_function=embeddings
)

C:\Users\Mayur kadam\AppData\Local\Temp\ipykernel_25468\3919521098.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  doc_serach=Chroma(


In [25]:
retrival=doc_serach.as_retriever(search_type="similarity",search_kargws={"k":5})

In [31]:
output=retrival.invoke("fever")

In [32]:
output

[Document(metadata={'page': 303, 'source': 'Data\\EN_GFARC_GUIDELINES_2020.pdf'}, page_content='In the early stages of fever, the person may have a high temperature but tell you they feel cold, or they may \nbe shivering or have chattering teeth or goose pimples. \nThe person may experience:\n• a high temperature above 38°C (100.4°F)\n• hot, flushed skin and sweating \n• headache or other aches and pains\n• loss of appetite\n• fatigue.\nFirst aid steps \n1. Advise the person to rest and dress lightly.\n2. Offer the person fluids to drink to replace lost fluids from sweating.'),
 Document(metadata={'page': 304, 'source': 'Data\\EN_GFARC_GUIDELINES_2020.pdf'}, page_content='In the early stages of fever, the person may have a high temperature but tell you they feel cold, or they may \nbe shivering or have chattering teeth or goose pimples. \nThe person may experience:\n• a high temperature above 38°C (100.4°F)\n• hot, flushed skin and sweating \n• headache or other aches and pains\n• loss

In [42]:
from langchain_core.prompts import ChatPromptTemplate 
from langchain_community.llms import Ollama 
from langchain.chains.retrieval import create_retrieval_chain 
from langchain.chains.combine_documents import create_stuff_documents_chain 


llm=Ollama(model="dolphin3") 

system_prompt=(
    "you are medical assistant chatbot" 
    "answer friendly and politely"
    "answer length maximum three or four dentences"
    "answer should be concise"
    "\n\n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages([
    ("system",system_prompt), 
    ("human","{input}")
])

In [45]:
question_answer=create_stuff_documents_chain(llm=llm,prompt=prompt)
rag_chain=create_retrieval_chain(retrival,question_answer)

In [46]:
result=rag_chain.invoke({"input":"can you have some remedy over fever?"})

In [48]:
result["answer"]

"Yes, in many cases medication isn't necessary. You can control a fever by giving paracetamol (acetaminophen) at the recommended dose or sponging with lukewarm water gently. Remember to monitor their condition closely and seek help immediately if there are additional concerning signs like a rash, change in mental status, difficulty breathing, severe abdominal pain, sensitivity to light, vomiting, or signs of shock."